In [1]:
from __future__ import print_function
import os
from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from models import G, D, weights_init
from data import get_training_set, get_test_set
import torch.backends.cudnn as cudnn
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
#torch.backends.cudnn.enabled = False


In [2]:
import numpy as np

In [3]:

batchSize=10
testBatchSize=10
nEpochs=500000
threads=2
lamb=100
seed=187
cuda=True
beta1=0.5
lr=0.0002
ndf=16
ngf=16
input_nc=3
output_nc=3

dataset='hdr'
dataroot='all_batched_dataset_256/mat_files/'
path=dataroot

import os
import scipy.io
ls=os.listdir(path)
#print ls
xs=[]
ys=[]
#for idx,i in enumerate(ls):
#    if(idx%100==0):
#        print ((idx,'ls'))
#    im1 = scipy.io.loadmat(path+i)
#    im1=im1['im']
#    xs.append(im1.shape[0])
#    ys.append(im1.shape[1])
s1=(256)
s2=(256)
print (s1)
print (s2)
imageSize=(s1,s2,3)

s1x=256
s2x=256
##################
ls=ls[0:100]

#####################
lls=len(ls)

import numpy as np
print (lls*3*s1*s2)
#X=np.zeros((lls,3,s1,s2))
#Y=np.zeros((lls,3,s1,s2))
X=np.random.rand(lls,3,s1,s2)
Y=np.random.rand(lls,3,s1x,s2x)

print ('phew')
count=0
import Image
for idx,i in enumerate(ls):
    li=len(i)
    i=i[0:li-4]
    im1 = scipy.io.loadmat(path+i+'.mat')
    im2=np.asarray(Image.open('all_batched_dataset_256/image_files/'+i+'.png'))    
    im2=im2/255.0;
    im1=im1['im']

    X[count,0,:,:]=im1[:    , : ,0]
    X[count,1,:,:]=im1[ :    , : ,1]
    X[count,2,:,:]=im1[:    , : ,2]

    Y[count,0, :    , : ]=im2[ :    , : ,0]
    Y[count,1, :    , : ]=im2[ :    , : ,1]
    Y[count,2, :    , : ]=im2[ :    , : ,2]
    count=count+1
    if(idx%1000==0):
        print (idx)
print(X.shape)


if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

cudnn.benchmark = True

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)




256
256
19660800
phew
0
(100, 3, 256, 256)


In [4]:
class ResnetGenerator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False, n_blocks=6, gpu_ids=[]):
        assert(n_blocks >= 0)
        super(ResnetGenerator, self).__init__()
        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf
        self.gpu_ids = gpu_ids

        model = [nn.Conv2d(input_nc, ngf, kernel_size=7, padding=3),
                 norm_layer(ngf, affine=True),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):
            mult = 2**i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3,
                                stride=2, padding=1),
                      norm_layer(ngf * mult * 2, affine=True),
                      nn.ReLU(True)]

        mult = 2**n_downsampling
        for i in range(n_blocks):
            model += [ResnetBlock(ngf * mult, 'zero', norm_layer=norm_layer, use_dropout=use_dropout)]

        for i in range(n_downsampling):
            mult = 2**(n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1),
                      norm_layer(int(ngf * mult / 2), affine=True),
                      nn.ReLU(True)]

        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=3)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, input):
        if self.gpu_ids and isinstance(input.data, torch.cuda.FloatTensor):
            return nn.parallel.data_parallel(self.model, input, self.gpu_ids)
        else:
            return self.model(input)


# Define a resnet block
class ResnetBlock(nn.Module):
    def __init__(self, dim, padding_type, norm_layer, use_dropout):
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout):
        conv_block = []
        p = 0
        # TODO: support padding types
        assert(padding_type == 'zero')
        p = 1

        # TODO: InstanceNorm
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p),
                       norm_layer(dim, affine=True),
                       nn.ReLU(True)]
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p),
                       norm_layer(dim, affine=True)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        out = x + self.conv_block(x)
        return out



In [5]:
class NLayerDiscriminator(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d, use_sigmoid=False, gpu_ids=[]):
        super(NLayerDiscriminator, self).__init__()
        self.gpu_ids = gpu_ids

        kw = 4
        padw = int(np.ceil((kw-1)/2))
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw),
            nn.LeakyReLU(0.2, True)
        ]

        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                                kernel_size=kw, stride=2, padding=padw),
                # TODO: use InstanceNorm
                norm_layer(ndf * nf_mult, affine=True),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                            kernel_size=kw, stride=1, padding=padw),
            # TODO: useInstanceNorm
            norm_layer(ndf * nf_mult, affine=True),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]

        if use_sigmoid:
            sequence += [nn.Sigmoid()]

        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        if len(self.gpu_ids)  and isinstance(input.data, torch.cuda.FloatTensor):
            return nn.parallel.data_parallel(self.model, input, self.gpu_ids)
        else:
            return self.model(input)


In [6]:

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1 or  classname.find('InstanceNorm2d') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def get_norm_layer(norm_type):
    if norm_type == 'batch':
        norm_layer = nn.BatchNorm2d
    elif norm_type == 'instance':
        norm_layer = nn.InstanceNorm2d
    else:
        print('normalization layer [%s] is not found' % norm)
    return norm_layer

def define_G(input_nc, output_nc, ngf,  norm='batch', use_dropout=False, gpu_ids=[]):
    netG = None
    norm_layer = get_norm_layer(norm_type=norm)
    netG = ResnetGenerator(input_nc, output_nc, ngf, norm_layer=norm_layer, use_dropout=use_dropout, n_blocks=9, gpu_ids=gpu_ids)
    netG.apply(weights_init)
    return netG


def define_D(input_nc, ndf,
             n_layers_D=3, norm='batch', use_sigmoid=False, gpu_ids=[]):
    netD = None
    norm_layer = get_norm_layer(norm_type=norm)
    netD = NLayerDiscriminator(input_nc, ndf, n_layers=3, norm_layer=norm_layer, use_sigmoid=use_sigmoid, gpu_ids=gpu_ids)
    netD.apply(weights_init)
    return netD


In [7]:

def get_batch_train():
    import numpy.random
    pos=numpy.random.randint(0,X.shape[0],(batchSize))
    x1=X[pos]
    y1=Y[pos]
    x1=torch.FloatTensor(x1)
    y1=torch.FloatTensor(y1)
    return (x1), (y1)


x1,y1=get_batch_train()
print('===> Building model')
netG = define_G(input_nc, output_nc, ngf)
netG.apply(weights_init)
netD = define_D(input_nc, output_nc, ndf)
netD.apply(weights_init)



===> Building model


NLayerDiscriminator (
  (model): Sequential (
    (0): Conv2d(3, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(3, 6, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(6, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(12, 24, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (11): Conv2d(24, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [8]:
criterion = nn.BCELoss()
criterion_l1 = nn.L1Loss()
criterion_mse = nn.MSELoss()

real_A = torch.FloatTensor(batchSize, input_nc, s1, s2)
fake_v = torch.FloatTensor(batchSize, input_nc, s1, s2)
real_B = torch.FloatTensor(batchSize, output_nc, s1x, s2x)
label = torch.FloatTensor(batchSize)
real_label = 1
fake_label = 0

if cuda:
    netD = netD.cuda()
    netG = netG.cuda()
    criterion = criterion.cuda()
    criterion_l1 = criterion_l1.cuda()
    criterion_mse = criterion_mse.cuda()
    real_A = real_A.cuda()
    real_B = real_B.cuda()
    fake_v = fake_v.cuda()
    
    label = label.cuda()


real_A = Variable(real_A)
fake_v = Variable(fake_v)
real_B = Variable(real_B)
label = Variable(label)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))



In [9]:

outf='output_dilated_true/'
def train(epoch):
    for iteration in range(10):        
        batchX,batchY=get_batch_train()
        batch=(batchX,batchY)
        print ('ga')
        ############################
        # (1) Update D network: maximize log(D(x,y)) + log(1 - D(x,G(x)))
        ###########################
        # train with real
        netD.volatile = False
        netD.zero_grad()
        real_a_cpu, real_b_cpu = batch[0], batch[1]
        #print(real_a_cpu)
        #print (real_a_cpu)
        #print (type(real_b_cpu))
        #print ((real_A.data))
        
        
        real_A.data.resize_(real_a_cpu.size()).copy_(real_a_cpu)
        real_B.data.resize_(real_b_cpu.size()).copy_(real_b_cpu)

        output = netD(torch.cat(  (real_A, real_B), 1))
        label.data.resize_(output.size()).fill_(real_label)
        err_d_real = criterion(output, label)
        err_d_real.backward()
        d_x_y = output.data.mean()

        # train with fake
        fake_b = netG(real_A)
        fake_v.data[:,:,5:s1-5,5:s2-5]=fake_b.data
        '''
        import numpy as np
        xxva2= np.zeros((batchSize,3,s1,s2))
        xxva=(fake_b.data.numpy())
        for va1 in range(batchSize):
            xxva2[va1,:,5:s1-5,5:s1-5]=xxva[va1,:,:,:]
            
        
        print (xxva2.shape)
        
        fake_b=Variable(torch.from_numpy(xxva2   ).float().cuda())
        '''
        
        output = netD(torch.cat((real_A, fake_v.detach()), 1))
        label.data.resize_(output.size()).fill_(fake_label)
        err_d_fake = criterion(output, label)
        err_d_fake.backward()
        d_x_gx = output.data.mean()

        err_d = (err_d_real + err_d_fake) / 2.0
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(x,G(x))) + L1(y,G(x))
        ###########################
        netG.zero_grad()
        netD.volatile = True
        output = netD(torch.cat((real_A, fake_v), 1))
        label.data.resize_(output.size()).fill_(real_label)
        err_g = criterion(output, label) + lamb * criterion_l1(fake_v, real_B)
        err_g.backward()
        d_x_gx_2 = output.data.mean()
        optimizerG.step()

        print("===> Epoch[{}]({}/{}): Loss_D: {:.4f} Loss_G: {:.4f} D(x): {:.4f} D(G(z)): {:.4f}/{:.4f}".format(
            epoch, iteration, 10, err_d.data[0], err_g.data[0], d_x_y, d_x_gx, d_x_gx_2))

        if(iteration==9 and epoch % 5==0):
            import numpy as np
            ss=Y.shape[0]
            xyvap=np.random.randint(ss)
            #vutils.save_image(Y[xyvap],
            #        '%s/real_samples.png' % outf)

            ddv=X[xyvap:xyvap+1]
            #print (ddv.shape)
            ddv2=Variable(torch.from_numpy(ddv).float().cuda())

            ddv3=Y[xyvap:xyvap+1]
            #print (ddv.shape)
            ddv4=Variable(torch.from_numpy(ddv3).float().cuda())
            #print (ddv2)
            fake = netG(ddv2)
            #print (fake.data)
            #print (ddv4)

            vutils.save_image(fake.data,
                    '%s/samples_epoch_%03d.png' % (outf, epoch) , normalize=False)
            vutils.save_image(ddv4.data,
                    '%s/samples_epoch_%03d.png' % (outf, epoch+1), normalize=False)




In [10]:


def test():
    avg_psnr = 0
    for batch in testing_data_loader:
        input, target = Variable(batch[0]), Variable(batch[1])
        if cuda:
            input = input.cuda()
            target = target.cuda()

        prediction = netG(input)
        mse = criterion_mse(prediction, target)
        psnr = 10 * log10(1 / mse.data[0])
        avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))


def checkpoint(epoch):
    if not os.path.exists("checkpoint"):
        os.mkdir("checkpoint")
    if not os.path.exists(os.path.join("checkpoint", dataset)):
        os.mkdir(os.path.join("checkpoint", dataset))
    net_g_model_out_path = "checkpoint/{}/netG_model_epoch_{}.pth".format(dataset, epoch)
    net_d_model_out_path = "checkpoint/{}/netD_model_epoch_{}.pth".format(dataset, epoch)
    torch.save(netG.state_dict(), net_g_model_out_path)
    torch.save(netD.state_dict(), net_d_model_out_path)
    print("Checkpoint saved to {}".format("checkpoint" + dataset))

for epoch in range(0, nEpochs + 1):
    print (epoch)
    train(epoch)
    #test()
    if epoch % 50 == 0:
        checkpoint(epoch)


0
ga


RuntimeError: CUDNN_STATUS_BAD_PARAM